**Remove_Background**

In [1]:
import requests
import os
from PIL import Image
from io import BytesIO

def remove_background(api_key, input_image_path, output_image_path):
    """
    Remove the background from an image using remove.bg API
    and save the output to the specified path.
    """
    try:
        # Open the input image
        with open(input_image_path, 'rb') as image_file:
            response = requests.post(
                'https://api.remove.bg/v1.0/removebg',
                files={'image_file': image_file},
                data={'size': 'auto'},
                headers={'X-Api-Key': api_key}
            )
        
        # Check if the API call was successful
        if response.status_code == 200:
            with open(output_image_path, 'wb') as output_file:
                output_file.write(response.content)
            print(f"✅ Background removed successfully! Image saved at {output_image_path}")
        else:
            print(f"❌ Error: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Replace 'your_api_key' with your actual remove.bg API key
api_key = "d1Jr7omr7SZZec67LYFbwSxP"  # <-- Replace with your API key from remove.bg

# Input and output file paths
input_image = "Original_img.jpg"   # Input file name (ensure it's in the same folder as main.py)
output_image = "Output_img.png"    # Output file name

# Call the function
remove_background(api_key, input_image, output_image)


❌ An error occurred: [Errno 2] No such file or directory: 'Original_img.jpg'


**solid_background**

In [ ]:
from PIL import Image

def add_solid_background(input_image_path, output_image_path, color=(255, 255, 255)):
    """
    Adds a solid color background to an image with transparency.
    :param input_image_path: Path to the image with a transparent background
    :param output_image_path: Path to save the new image with a solid background
    :param color: Solid color for the background (default is white - (255, 255, 255))
    """
    try:
        # Open the input image
        product_image = Image.open(input_image_path).convert("RGBA")
        
        # Create a new image with the solid color background
        solid_background = Image.new("RGBA", product_image.size, color)
        
        # Composite the product image onto the solid color background
        combined_image = Image.alpha_composite(solid_background, product_image)
        
        # Convert to RGB and save as JPG (to remove alpha channel)
        combined_image = combined_image.convert("RGB")
        combined_image.save(output_image_path, "JPEG")
        
        print(f"✅ Solid color background added! Image saved as {output_image_path}")
    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Usage
input_image = "Output_img.png"         # Input image with transparent background
output_image = "Solid_bg_output.jpg"   # Output image path
solid_color = (255, 255, 255)          # White background (RGB: 255, 255, 255)

add_solid_background(input_image, output_image, solid_color)




**gradient_background**

In [ ]:
from PIL import Image, ImageDraw

def add_gradient_background(input_image_path, output_image_path, top_color=(255, 0, 0), bottom_color=(255, 255, 255)):
    """
    Adds a vertical gradient background to an image with transparency.
    :param input_image_path: Path to the image with a transparent background
    :param output_image_path: Path to save the new image with the gradient background
    :param top_color: RGB color at the top of the gradient
    :param bottom_color: RGB color at the bottom of the gradient
    """
    try:
        # Open the input image
        product_image = Image.open(input_image_path).convert("RGBA")
        width, height = product_image.size
        
        # Create a new image for the gradient background
        gradient = Image.new("RGBA", (width, height), top_color)
        draw = ImageDraw.Draw(gradient)
        
        # Draw the gradient
        for y in range(height):
            # Calculate intermediate color for each row
            r = top_color[0] + (bottom_color[0] - top_color[0]) * y // height
            g = top_color[1] + (bottom_color[1] - top_color[1]) * y // height
            b = top_color[2] + (bottom_color[2] - top_color[2]) * y // height
            draw.line([(0, y), (width, y)], fill=(r, g, b))
        
        # Composite the product image onto the gradient background
        combined_image = Image.alpha_composite(gradient, product_image)
        
        # Convert to RGB and save as JPG
        combined_image = combined_image.convert("RGB")
        combined_image.save(output_image_path, "JPEG")
        
        print(f"✅ Gradient background added! Image saved as {output_image_path}")
    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Usage
input_image = "Output_img.png"         # Input image with transparent background
output_image = "Gradient_bg_output.jpg" # Output image path
top_color = (255, 0, 0)                # Red at the top
bottom_color = (255, 255, 255)         # White at the bottom

add_gradient_background(input_image, output_image, top_color, bottom_color)


In [ ]:
from PIL import Image, ImageFilter, ImageOps

def add_shadow(input_image_path, output_image_path, shadow_offset=(10, 10), shadow_blur=10, shadow_color=(0, 0, 0, 150)):
    """
    Adds a soft drop shadow to an image with transparency.
    :param input_image_path: Path to the input image (transparent background)
    :param output_image_path: Path to save the shadowed image
    :param shadow_offset: (x, y) tuple specifying the shadow offset
    :param shadow_blur: Radius for the Gaussian blur to soften the shadow
    :param shadow_color: RGBA color for the shadow (default is semi-transparent black)
    """
    try:
        # Open the input image
        product_image = Image.open(input_image_path).convert("RGBA")
        width, height = product_image.size
        
        # Create an empty image for the shadow
        shadow = Image.new("RGBA", (width, height), (0, 0, 0, 0))
        
        # Create the shadow layer
        shadow_layer = product_image.split()[3]  # Extract the alpha channel (transparency mask)
        shadow_layer = ImageOps.offset(shadow_layer, shadow_offset[0], shadow_offset[1])
        shadow_layer = shadow_layer.filter(ImageFilter.GaussianBlur(shadow_blur))
        
        # Paste the shadow onto the shadow image
        shadow.paste(shadow_color, (0, 0), shadow_layer)
        
        # Combine the shadow with the original product image
        combined = Image.alpha_composite(shadow, product_image)
        
        # Convert to RGB and save as JPEG
        combined = combined.convert("RGB")
        combined.save(output_image_path, "JPEG")
        
        print(f"✅ Shadow added successfully! Image saved as {output_image_path}")
    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Usage
input_image = "Output_img.png"          # Input image with transparent background
output_image = "Shadowed_output.jpg"    # Output image path
shadow_offset = (15, 15)                # Shadow offset (x, y)
shadow_blur = 15                        # Shadow softness (Gaussian blur radius)
shadow_color = (0, 0, 0, 120)           # Shadow color (semi-transparent black)

add_shadow(input_image, output_image, shadow_offset, shadow_blur, shadow_color)

**gradient_background_with_shadow**

In [ ]:
from PIL import Image, ImageDraw, ImageFilter

def add_gradient_background_with_shadow(input_image_path, output_image_path, top_color=(255, 0, 0), bottom_color=(255, 255, 255),
                                        shadow_offset=(15, 15), shadow_blur=15, shadow_color=(0, 0, 0, 120)):
    """
    Combines a shadowed product image with a gradient background.
    :param input_image_path: Path to the input image (transparent background)
    :param output_image_path: Path to save the new image
    :param top_color: RGB color at the top of the gradient
    :param bottom_color: RGB color at the bottom of the gradient
    :param shadow_offset: (x, y) tuple specifying the shadow offset
    :param shadow_blur: Radius for the Gaussian blur to soften the shadow
    :param shadow_color: RGBA color for the shadow (semi-transparent black)
    """
    try:
        # Open the input image
        product_image = Image.open(input_image_path).convert("RGBA")
        width, height = product_image.size

        # Step 1: Create a gradient background
        gradient = Image.new("RGBA", (width, height), top_color)
        draw = ImageDraw.Draw(gradient)

        for y in range(height):
            # Calculate the gradient color for each row
            r = top_color[0] + (bottom_color[0] - top_color[0]) * y // height
            g = top_color[1] + (bottom_color[1] - top_color[1]) * y // height
            b = top_color[2] + (bottom_color[2] - top_color[2]) * y // height
            draw.line([(0, y), (width, y)], fill=(r, g, b))

        # Step 2: Create the shadow
        shadow = Image.new("RGBA", (width + abs(shadow_offset[0]), height + abs(shadow_offset[1])), (0, 0, 0, 0))
        shadow_mask = product_image.split()[3]  # Alpha channel (transparency mask)
        shadow_mask = shadow_mask.filter(ImageFilter.GaussianBlur(shadow_blur))
        shadow.paste(shadow_color, (max(shadow_offset[0], 0), max(shadow_offset[1], 0)), shadow_mask)

        # Step 3: Paste the product image and shadow onto the gradient background
        combined = Image.alpha_composite(gradient, shadow)  # Add shadow to the gradient
        combined.paste(product_image, (abs(min(shadow_offset[0], 0)), abs(min(shadow_offset[1], 0))), product_image)

        # Convert to RGB and save as JPEG
        combined = combined.convert("RGB")
        combined.save(output_image_path, "JPEG")

        print(f"✅ Shadow combined with gradient background! Image saved as {output_image_path}")
    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Usage
input_image = "Output_img.png"                # Input image with transparent background
output_image = "Shadow_Gradient_Output.jpg"   # Output file name
top_color = (255, 0, 0)                       # Red at the top
bottom_color = (255, 255, 255)                # White at the bottom
shadow_offset = (15, 15)                      # Shadow offset (x, y)
shadow_blur = 15                              # Shadow softness
shadow_color = (0, 0, 0, 120)                 # Semi-transparent black shadow

add_gradient_background_with_shadow(input_image, output_image, top_color, bottom_color,
                                    shadow_offset, shadow_blur, shadow_color)


**studio_background**

In [ ]:
from PIL import Image, ImageDraw, ImageFilter

def add_studio_background(input_image_path, output_image_path, light_center=(255, 255, 255), edge_color=(200, 200, 200),
                          shadow_offset=(15, 15), shadow_blur=20, shadow_color=(0, 0, 0, 100)):
    """
    Adds a soft studio background with lighting and shadow to the product image.
    :param input_image_path: Path to the input image (transparent background)
    :param output_image_path: Path to save the new image
    :param light_center: RGB color for the center of the studio light
    :param edge_color: RGB color for the edges of the studio background
    :param shadow_offset: (x, y) tuple specifying the shadow offset
    :param shadow_blur: Radius for the Gaussian blur to soften the shadow
    :param shadow_color: RGBA color for the shadow (default is semi-transparent black)
    """
    try:
        # Open the input image
        product_image = Image.open(input_image_path).convert("RGBA")
        width, height = product_image.size

        # Step 1: Create a radial gradient for studio lighting
        studio_background = Image.new("RGBA", (width, height), edge_color)
        draw = ImageDraw.Draw(studio_background)

        # Draw radial gradient manually
        max_radius = max(width, height) // 2
        for i in range(max_radius, 0, -1):
            alpha = int(255 * (1 - i / max_radius))  # Calculate opacity
            fill_color = tuple([
                int(light_center[j] * (alpha / 255) + edge_color[j] * (1 - alpha / 255)) for j in range(3)
            ])
            draw.ellipse([
                (width // 2 - i, height // 2 - i),
                (width // 2 + i, height // 2 + i)
            ], fill=fill_color)

        # Step 2: Create the shadow
        shadow = Image.new("RGBA", (width, height), (0, 0, 0, 0))
        shadow_mask = product_image.split()[3]  # Alpha channel (transparency mask)
        shadow_mask = shadow_mask.filter(ImageFilter.GaussianBlur(shadow_blur))
        shadow.paste(shadow_color, shadow_offset, shadow_mask)

        # Step 3: Combine the background, shadow, and product image
        combined = Image.alpha_composite(studio_background, shadow)  # Add shadow to studio background
        combined = Image.alpha_composite(combined, product_image)  # Add product image on top

        # Convert to RGB and save as JPEG
        combined = combined.convert("RGB")
        combined.save(output_image_path, "JPEG")

        print(f"✅ Studio setting background added! Image saved as {output_image_path}")
    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Usage
input_image = "Output_img.png"                # Input image with transparent background
output_image = "Studio_Background_Output.jpg" # Output file name
light_center = (255, 255, 255)                # Bright white light center
edge_color = (200, 200, 200)                  # Soft grey edges
shadow_offset = (15, 15)                      # Shadow offset (x, y)
shadow_blur = 20                              # Shadow softness
shadow_color = (0, 0, 0, 100)                 # Semi-transparent black shadow

add_studio_background(input_image, output_image, light_center, edge_color,
                      shadow_offset, shadow_blur, shadow_color)


**lifestyle_context**

In [ ]:
from PIL import Image

def add_lifestyle_context(product_image_path, lifestyle_image_path, output_image_path, position=(50, 50)):
    """
    Places the product image onto a lifestyle background.
    :param product_image_path: Path to the product image (transparent background)
    :param lifestyle_image_path: Path to the lifestyle background image
    :param output_image_path: Path to save the final combined image
    :param position: (x, y) position to place the product on the background
    """
    try:
        # Open the lifestyle background image
        lifestyle_background = Image.open(lifestyle_image_path).convert("RGBA")
        
        # Open the product image with transparency
        product_image = Image.open(product_image_path).convert("RGBA")
        
        # Resize the product image if needed
        product_image = product_image.resize((int(lifestyle_background.width * 0.4),  # Scale to 40% of width
                                             int(lifestyle_background.height * 0.4)), 
                                             Image.LANCZOS)
        
        # Paste the product image onto the background at the specified position
        lifestyle_background.paste(product_image, position, product_image)
        
        # Convert to RGB and save as a final image
        lifestyle_background = lifestyle_background.convert("RGB")
        lifestyle_background.save(output_image_path, "JPEG")
        
        print(f"✅ Lifestyle context added successfully! Image saved as {output_image_path}")
    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Usage
product_image_path = "Output_img.png"                # Product image with transparency
lifestyle_image_path = "Lifestyle_bg.jpg"           # Lifestyle background image (e.g., kitchen, desk)
output_image_path = "Lifestyle_Context_Output.jpg"  # Output file name
position = (50, 200)                                # Position of the product on the background (x, y)

add_lifestyle_context(product_image_path, lifestyle_image_path, output_image_path, position)

**reflection_and_shadow**

In [ ]:
from PIL import Image, ImageDraw, ImageFilter

def add_reflection_and_shadow(product_image_path, background_image_path, output_image_path,
                              shadow_offset=(20, 20), shadow_blur=20, shadow_opacity=120,
                              reflection_opacity=150, reflection_scale=0.4):
    """
    Adds realistic shadow and reflection to a product image placed on a background.
    :param product_image_path: Path to the transparent product image
    :param background_image_path: Path to the background image
    :param output_image_path: Path to save the final image
    :param shadow_offset: Tuple (x, y) for shadow position
    :param shadow_blur: Radius of the Gaussian blur for soft shadows
    :param shadow_opacity: Opacity of the shadow (0-255)
    :param reflection_opacity: Opacity of the reflection (0-255)
    :param reflection_scale: Scale of the reflection relative to the product size
    """
    try:
        # Open product and background images
        product_image = Image.open(product_image_path).convert("RGBA")
        background = Image.open(background_image_path).convert("RGBA")

        # Step 1: Resize background to match product image dimensions
        background_resized = background.resize(product_image.size, Image.LANCZOS)

        # Step 2: Create shadow
        shadow = Image.new("RGBA", product_image.size, (0, 0, 0, 0))
        shadow_mask = product_image.split()[3]  # Alpha channel
        shadow_mask = shadow_mask.filter(ImageFilter.GaussianBlur(shadow_blur))
        shadow.paste((0, 0, 0, shadow_opacity), shadow_offset, shadow_mask)

        # Step 3: Add shadow to the background
        combined = Image.alpha_composite(background_resized, shadow)

        # Step 4: Add product image on top of the shadow
        combined.paste(product_image, (0, 0), product_image)

        # Step 5: Create reflection
        reflection = product_image.transpose(Image.FLIP_TOP_BOTTOM).resize(
            (int(product_image.width), int(product_image.height * reflection_scale)), Image.LANCZOS
        )
        reflection_mask = reflection.split()[3].point(lambda p: p * (reflection_opacity / 255))
        combined.paste(reflection, (0, product_image.height - int(product_image.height * (1 - reflection_scale))), mask=reflection_mask)

        # Step 6: Save the final image
        combined.convert("RGB").save(output_image_path, "JPEG")
        print(f"✅ Realistic image with shadow and reflection saved at: {output_image_path}")

    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Usage
product_image_path = "Output_img.png"                 # Transparent product image
background_image_path = "Lifestyle_bg.jpg"           # Background image (studio/lifestyle)
output_image_path = "Realistic_Depth_Output.jpg"     # Final output image

add_reflection_and_shadow(product_image_path, background_image_path, output_image_path)


**realistic_depth**

In [ ]:
from PIL import Image, ImageDraw, ImageFilter, ImageChops

def add_realistic_depth(product_image_path, background_image_path, output_image_path,
                        shadow_offset=(20, 20), shadow_blur=25, shadow_opacity=100,
                        reflection_opacity=120, reflection_fade=True, reflection_scale=0.4):
    """
    Adds a soft shadow and a realistic reflection with fade-out to a product image on a background.
    :param product_image_path: Path to the transparent product image
    :param background_image_path: Path to the background image
    :param output_image_path: Path to save the final image
    :param shadow_offset: Offset (x, y) for shadow placement
    :param shadow_blur: Gaussian blur for softening shadow
    :param shadow_opacity: Opacity of the shadow (0-255)
    :param reflection_opacity: Opacity of the reflection (0-255)
    :param reflection_fade: Whether to apply a gradual fade to the reflection
    :param reflection_scale: Scale of the reflection height relative to the product image
    """
    try:
        # Open product and background images
        product_image = Image.open(product_image_path).convert("RGBA")
        background = Image.open(background_image_path).convert("RGBA")

        # Resize background to match product image dimensions
        background_resized = background.resize(product_image.size, Image.LANCZOS)

        # Step 1: Create soft shadow
        shadow = Image.new("RGBA", product_image.size, (0, 0, 0, 0))
        shadow_mask = product_image.split()[3].filter(ImageFilter.GaussianBlur(shadow_blur))
        shadow.paste((0, 0, 0, shadow_opacity), shadow_offset, shadow_mask)

        # Step 2: Composite shadow onto background
        combined = Image.alpha_composite(background_resized, shadow)

        # Step 3: Paste product image
        combined.paste(product_image, (0, 0), product_image)

        # Step 4: Add reflection
        reflection = product_image.transpose(Image.FLIP_TOP_BOTTOM).resize(
            (product_image.width, int(product_image.height * reflection_scale)), Image.LANCZOS
        )
        reflection_mask = reflection.split()[3].point(lambda p: p * (reflection_opacity / 255))

        # Gradual fade for the reflection
        if reflection_fade:
            fade_mask = Image.new("L", (reflection.width, reflection.height), 0)
            draw = ImageDraw.Draw(fade_mask)
            for y in range(reflection.height):
                fade_value = int(255 * (1 - y / reflection.height))
                draw.line([(0, y), (reflection.width, y)], fill=fade_value)
            reflection_mask = ImageChops.multiply(reflection_mask, fade_mask)

        combined.paste(reflection, (0, product_image.height - int(product_image.height * (1 - reflection_scale))), mask=reflection_mask)

        # Step 5: Save the final image
        combined.convert("RGB").save(output_image_path, "JPEG")
        print(f"✅ Refined image with realistic shadow and reflection saved at: {output_image_path}")

    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Usage
product_image_path = "Output_img.png"                 # Transparent product image
background_image_path = "Lifestyle_bg.jpg"           # Background image
output_image_path = "Refined_Realistic_Output.jpg"   # Output file path

add_realistic_depth(product_image_path, background_image_path, output_image_path)


**realistic_reflection_and_shadow**

In [ ]:
from PIL import Image, ImageDraw, ImageFilter, ImageChops

def add_realistic_reflection_and_shadow(product_image_path, background_image_path, output_image_path,
                                        shadow_offset=(20, 20), shadow_blur=30, shadow_opacity=100,
                                        reflection_opacity=120, reflection_scale=0.4, reflection_offset=30,
                                        add_gloss=True):
    """
    Adds a soft shadow, refined reflection, and optional glossy highlight to a product image.
    :param product_image_path: Path to the transparent product image
    :param background_image_path: Path to the background image
    :param output_image_path: Path to save the final image
    :param shadow_offset: Offset (x, y) for shadow placement
    :param shadow_blur: Gaussian blur radius for soft shadows
    :param shadow_opacity: Opacity of the shadow (0-255)
    :param reflection_opacity: Opacity of the reflection (0-255)
    :param reflection_scale: Scale of the reflection height relative to the product image
    :param reflection_offset: Offset to lower the reflection for separation
    :param add_gloss: Whether to add a glossy highlight for realism
    """
    try:
        # Open the product image and background
        product_image = Image.open(product_image_path).convert("RGBA")
        background = Image.open(background_image_path).convert("RGBA")

        # Resize the background to match product image dimensions
        background_resized = background.resize(product_image.size, Image.LANCZOS)

        # Step 1: Create soft shadow
        shadow = Image.new("RGBA", product_image.size, (0, 0, 0, 0))
        shadow_mask = product_image.split()[3].filter(ImageFilter.GaussianBlur(shadow_blur))
        shadow.paste((0, 0, 0, shadow_opacity), shadow_offset, shadow_mask)

        # Step 2: Composite shadow onto the background
        combined = Image.alpha_composite(background_resized, shadow)

        # Step 3: Paste product image on top of shadow
        combined.paste(product_image, (0, 0), product_image)

        # Step 4: Create and add reflection
        reflection = product_image.transpose(Image.FLIP_TOP_BOTTOM).resize(
            (product_image.width, int(product_image.height * reflection_scale)), Image.LANCZOS
        )
        reflection_mask = reflection.split()[3].point(lambda p: p * (reflection_opacity / 255))

        # Apply a fade to the reflection using a gradient
        fade_mask = Image.new("L", (reflection.width, reflection.height), 0)
        draw = ImageDraw.Draw(fade_mask)
        for y in range(reflection.height):
            fade_value = int(255 * (1 - y / reflection.height))
            draw.line([(0, y), (reflection.width, y)], fill=fade_value)
        reflection_mask = ImageChops.multiply(reflection_mask, fade_mask)

        # Position the reflection slightly below the product image
        reflection_y = product_image.height + reflection_offset
        combined.paste(reflection, (0, reflection_y), mask=reflection_mask)

        # Step 5: Add a subtle glossy highlight
        if add_gloss:
            gloss = Image.new("RGBA", product_image.size, (255, 255, 255, 0))
            gloss_draw = ImageDraw.Draw(gloss)
            gloss_draw.ellipse(
                [product_image.width // 3, product_image.height // 5, product_image.width, product_image.height // 2],
                fill=(255, 255, 255, 50)
            )
            combined = Image.alpha_composite(combined, gloss)

        # Step 6: Save the final image
        combined.convert("RGB").save(output_image_path, "JPEG")
        print(f"✅ Refined image with shadow, reflection, and gloss saved at: {output_image_path}")

    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Usage
product_image_path = "Output_img.png"                  # Transparent product image
background_image_path = "Lifestyle_bg.jpg"            # Background image
output_image_path = "Final_Realistic_Output.jpg"      # Output file path

add_realistic_reflection_and_shadow(product_image_path, background_image_path, output_image_path)


**final_polish**

In [ ]:
from PIL import Image, ImageDraw, ImageFilter, ImageChops

def add_final_polish(product_image_path, background_color, output_image_path,
                     shadow_offset=(20, 20), shadow_blur=50, shadow_opacity=80,
                     reflection_opacity=90, reflection_scale=0.4, reflection_offset=30,
                     add_gloss=True, blur_reflection=True):
    """
    Enhances product image with a cleaner background, soft shadow, blurred reflection, and subtle gloss.
    :param product_image_path: Path to the transparent product image
    :param background_color: Tuple (R, G, B) for a clean background
    :param output_image_path: Path to save the final image
    :param shadow_offset: Offset (x, y) for shadow placement
    :param shadow_blur: Gaussian blur radius for softening shadow
    :param shadow_opacity: Opacity of the shadow (0-255)
    :param reflection_opacity: Opacity of the reflection (0-255)
    :param reflection_scale: Scale of the reflection height relative to the product image
    :param reflection_offset: Offset to lower the reflection for separation
    :param add_gloss: Whether to add a glossy highlight for realism
    :param blur_reflection: Whether to blur the reflection slightly
    """
    try:
        # Open the product image
        product_image = Image.open(product_image_path).convert("RGBA")
        width, height = product_image.size

        # Step 1: Create a clean background
        background = Image.new("RGBA", (width, height), background_color)

        # Step 2: Create soft shadow
        shadow = Image.new("RGBA", (width, height), (0, 0, 0, 0))
        shadow_mask = product_image.split()[3]  # Alpha channel
        shadow_blur_filter = shadow_mask.filter(ImageFilter.GaussianBlur(shadow_blur))
        shadow.paste((0, 0, 0, shadow_opacity), shadow_offset, shadow_blur_filter)

        # Step 3: Composite shadow onto the clean background
        combined = Image.alpha_composite(background, shadow)

        # Step 4: Paste the product image
        combined.paste(product_image, (0, 0), product_image)

        # Step 5: Add reflection with blur
        reflection = product_image.transpose(Image.FLIP_TOP_BOTTOM).resize(
            (product_image.width, int(product_image.height * reflection_scale)), Image.LANCZOS
        )
        reflection_mask = reflection.split()[3].point(lambda p: p * (reflection_opacity / 255))

        # Apply a subtle blur to the reflection
        if blur_reflection:
            reflection = reflection.filter(ImageFilter.GaussianBlur(5))

        # Position the reflection slightly lower
        reflection_y = height - int(height * (1 - reflection_scale)) + reflection_offset
        combined.paste(reflection, (0, reflection_y), mask=reflection_mask)

        # Step 6: Add a subtle glossy highlight
        if add_gloss:
            gloss = Image.new("RGBA", (width, height), (255, 255, 255, 0))
            gloss_draw = ImageDraw.Draw(gloss)
            gloss_draw.ellipse(
                [width // 4, height // 6, width * 3 // 4, height // 3],
                fill=(255, 255, 255, 40)
            )
            combined = Image.alpha_composite(combined, gloss)

        # Step 7: Save the final polished image
        combined.convert("RGB").save(output_image_path, "JPEG")
        print(f"✅ Final polished image saved at: {output_image_path}")

    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Usage
product_image_path = "Output_img.png"                    # Transparent product image
background_color = (255, 255, 255)                      # Clean white background
output_image_path = "Final_Polished_Output.jpg"          # Output file path

add_final_polish(
    product_image_path=product_image_path,
    background_color=background_color,
    output_image_path=output_image_path,
    shadow_blur=50,              # Increase blur for softer shadows
    shadow_opacity=80,           # Reduce shadow intensity
    reflection_opacity=90,       # Lighten the reflection
    reflection_scale=0.4,        # Reflection height scale
    reflection_offset=20,        # Slight reflection offset
    add_gloss=True,              # Add subtle gloss
    blur_reflection=True         # Blur reflection slightly
)


**lifestyle_context**

In [ ]:
from PIL import Image

def add_lifestyle_context(product_image_path, lifestyle_image_path, output_image_path, position=(50, 50)):
    """
    Places the product image onto a lifestyle background.
    :param product_image_path: Path to the product image (transparent background)
    :param lifestyle_image_path: Path to the lifestyle background image
    :param output_image_path: Path to save the final combined image
    :param position: (x, y) position to place the product on the background
    """
    try:
        # Open the lifestyle background image
        lifestyle_background = Image.open(lifestyle_image_path).convert("RGBA")
        
        # Open the product image with transparency
        product_image = Image.open(product_image_path).convert("RGBA")
        
        # Resize the product image if needed
        product_image = product_image.resize((int(lifestyle_background.width * 0.4),  # Scale to 40% of width
                                             int(lifestyle_background.height * 0.4)), 
                                             Image.LANCZOS)
        
        # Paste the product image onto the background at the specified position
        lifestyle_background.paste(product_image, position, product_image)
        
        # Convert to RGB and save as a final image
        lifestyle_background = lifestyle_background.convert("RGB")
        lifestyle_background.save(output_image_path, "JPEG")
        
        print(f"✅ Lifestyle context added successfully! Image saved as {output_image_path}")
    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Usage
product_image_path = "Output_img.png"                # Product image with transparency
lifestyle_image_path = "Lifestyle_bg.jpg"           # Lifestyle background image (e.g., kitchen, desk)
output_image_path = "Lifestyle_Context_Output.jpg"  # Output file name
position = (50, 200)                                # Position of the product on the background (x, y)

add_lifestyle_context(product_image_path, lifestyle_image_path, output_image_path, position)


**solid_background**

In [ ]:
from PIL import Image, ImageDraw, ImageFilter
import os

# Function to add a solid color background
def add_solid_background(product_image, output_path, color=(255, 255, 255)):
    background = Image.new("RGBA", product_image.size, color)
    combined = Image.alpha_composite(background, product_image)
    combined.convert("RGB").save(output_path, "JPEG")
    print(f"✅ Solid background saved at: {output_path}")

# Function to add a gradient background
def add_gradient_background(product_image, output_path, top_color=(255, 0, 0), bottom_color=(255, 255, 255)):
    width, height = product_image.size
    gradient = Image.new("RGBA", (width, height), bottom_color)
    draw = ImageDraw.Draw(gradient)
    for y in range(height):
        r = top_color[0] + (bottom_color[0] - top_color[0]) * y // height
        g = top_color[1] + (bottom_color[1] - top_color[1]) * y // height
        b = top_color[2] + (bottom_color[2] - top_color[2]) * y // height
        draw.line([(0, y), (width, y)], fill=(r, g, b))
    combined = Image.alpha_composite(gradient, product_image)
    combined.convert("RGB").save(output_path, "JPEG")
    print(f"✅ Gradient background saved at: {output_path}")

# Function to add a simple lifestyle context
def add_lifestyle_background(product_image, lifestyle_image_path, output_path, position=(50, 50)):
    lifestyle_image = Image.open(lifestyle_image_path).convert("RGBA")
    resized_product = product_image.resize((int(lifestyle_image.width * 0.4), int(lifestyle_image.height * 0.4)), Image.LANCZOS)
    lifestyle_image.paste(resized_product, position, resized_product)
    lifestyle_image.convert("RGB").save(output_path, "JPEG")
    print(f"✅ Lifestyle background saved at: {output_path}")

# Function to add a studio setting background
def add_studio_background(product_image, output_path, light_center=(255, 255, 255), edge_color=(200, 200, 200)):
    width, height = product_image.size
    background = Image.new("RGBA", (width, height), edge_color)
    draw = ImageDraw.Draw(background)
    for i in range(width // 2, 0, -1):
        alpha = int(255 * (1 - i / (width // 2)))
        fill_color = tuple([int(light_center[j] * (alpha / 255) + edge_color[j] * (1 - alpha / 255)) for j in range(3)])
        draw.ellipse([i, i, width - i, height - i], fill=fill_color)
    combined = Image.alpha_composite(background, product_image)
    combined.convert("RGB").save(output_path, "JPEG")
    print(f"✅ Studio background saved at: {output_path}")

# Main function to generate all variations
def generate_image_variations(product_image_path, lifestyle_image_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    # Load the product image with transparency
    product_image = Image.open(product_image_path).convert("RGBA")
    
    # Generate solid color background
    add_solid_background(product_image, os.path.join(output_dir, "solid_background.jpg"), color=(255, 0, 0))
    
    # Generate gradient background
    add_gradient_background(product_image, os.path.join(output_dir, "gradient_background.jpg"),
                            top_color=(255, 0, 0), bottom_color=(255, 255, 255))
    
    # Generate lifestyle context background
    add_lifestyle_background(product_image, lifestyle_image_path, 
                             os.path.join(output_dir, "lifestyle_background.jpg"), position=(100, 100))
    
    # Generate studio setting background
    add_studio_background(product_image, os.path.join(output_dir, "studio_background.jpg"),
                          light_center=(255, 255, 255), edge_color=(200, 200, 200))

    print("🎉 All background variations have been generated successfully!")

# Example usage
product_image_path = "Output_img.png"                 # Transparent product image
lifestyle_image_path = "Lifestyle_bg.jpg"             # Lifestyle background image
output_directory = "background_variations"            # Directory to save outputs

generate_image_variations(product_image_path, lifestyle_image_path, output_directory)


**load_font-Code to Generate Multiple Variations**

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import os

def load_font(font_path, size):
    """Load custom font or fallback to default."""
    try:
        return ImageFont.truetype(font_path, size)
    except:
        print("⚠️ Font not found, falling back to default font.")
        return ImageFont.load_default()

def generate_variations(product_image_path, output_dir, font_path, variations):
    """Generate multiple text and banner variations."""
    os.makedirs(output_dir, exist_ok=True)

    for idx, var in enumerate(variations):
        try:
            image = Image.open(product_image_path).convert("RGBA")
            draw = ImageDraw.Draw(image)

            # Load fonts
            primary_font = load_font(font_path, var["primary_size"])
            support_font = load_font(font_path, var["support_size"])

            # Add banner
            (banner_left, banner_top), (banner_right, banner_bottom) = var["banner_coords"]
            draw.rectangle([banner_left, banner_top, banner_right, banner_bottom], fill=var["banner_color"])

            # Calculate primary text position
            banner_width = banner_right - banner_left
            banner_height = banner_bottom - banner_top

            primary_bbox = draw.textbbox((0, 0), var["primary_text"], font=primary_font)
            primary_x = banner_left + (banner_width - (primary_bbox[2] - primary_bbox[0])) // 2
            primary_y = banner_top + (banner_height // 4) - ((primary_bbox[3] - primary_bbox[1]) // 2)

            # Calculate support text position
            support_bbox = draw.textbbox((0, 0), var["support_text"], font=support_font)
            support_x = banner_left + (banner_width - (support_bbox[2] - support_bbox[0])) // 2
            support_y = primary_y + (primary_bbox[3] - primary_bbox[1]) + 10

            # Draw text
            draw.text((primary_x, primary_y), var["primary_text"], font=primary_font, fill=var["primary_color"])
            draw.text((support_x, support_y), var["support_text"], font=support_font, fill=var["support_color"])

            # Save the variation
            output_path = os.path.join(output_dir, f"variation_{idx+1}.jpg")
            image.convert("RGB").save(output_path, "JPEG")
            print(f"✅ Saved: {output_path}")

        except Exception as e:
            print(f"❌ Error with variation {idx+1}: {e}")

# Configuration
product_image_path = "Compliant_Final_Output.jpg"  # Input product image
output_directory = "ab_testing_variations"         # Output folder
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"  # Path to system font

# Define multiple variations
variations = [
    {
        "primary_text": "50% OFF",
        "primary_size": 100,
        "primary_color": "white",

        "support_text": "Limited Time Offer",
        "support_size": 50,
        "support_color": "yellow",

        "banner_coords": [(50, 50), (700, 200)],
        "banner_color": "red"
    },
    {
        "primary_text": "Exclusive Deal!",
        "primary_size": 80,
        "primary_color": "black",

        "support_text": "Shop Now!",
        "support_size": 50,
        "support_color": "white",

        "banner_coords": [(50, 400), (700, 550)],
        "banner_color": "gold"
    },
    {
        "primary_text": "New Arrivals",
        "primary_size": 90,
        "primary_color": "yellow",

        "support_text": "Don't Miss Out!",
        "support_size": 50,
        "support_color": "black",

        "banner_coords": [(50, 800), (700, 950)],
        "banner_color": "blue"
    }
]

# Generate variations
generate_variations(product_image_path, output_directory, font_path, variations)

**Create Hero Image**

In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageOps
import os

def create_hero_image(product_path, background_path, output_dir, text_elements, platform_sizes):
    """
    Combines product, text, and background into a hero image.

    Args:
    - product_path: Path to the product image.
    - background_path: Path to the background image.
    - output_dir: Directory to save outputs.
    - text_elements: Dictionary of text details.
    - platform_sizes: List of (width, height) tuples for platform-specific outputs.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Load images
    try:
        product_img = Image.open(product_path).convert("RGBA")
        background_img = Image.open(background_path).convert("RGBA")
    except FileNotFoundError as e:
        print(f"❌ Error: {e}")
        return

    for idx, (width, height) in enumerate(platform_sizes):
        # Resize background
        bg_resized = background_img.resize((width, height), Image.LANCZOS)

        # Position product in center
        product_resized = product_img.resize((int(width * 0.4), int(height * 0.4)))
        pos_x = (bg_resized.width - product_resized.width) // 2
        pos_y = (bg_resized.height - product_resized.height) // 2

        # Overlay product
        bg_resized.paste(product_resized, (pos_x, pos_y), product_resized)

        # Add text
        draw = ImageDraw.Draw(bg_resized)

        # Font handling with fallback
        try:
            font_path = "/System/Library/Fonts/Supplemental/Helvetica.ttc"  # MacOS default
            font = ImageFont.truetype(font_path, size=int(height * 0.05))
        except OSError:
            print("⚠️ Font not found, using default font.")
            font = ImageFont.load_default()

        # Draw text elements
        for text, position in text_elements.items():
            draw.text(position, text, font=font, fill="white")

        # Save output
        output_path = os.path.join(output_dir, f"hero_variation_{idx + 1}.png")
        bg_resized.save(output_path)
        print(f"✅ Hero image saved: {output_path}")

# Configuration
product_image = "product_with_transparency.png"  # Replace with your product image path
background_image = "gradient_bg.jpg"  # Replace with desired background image path
output_folder = "hero_images"

# Text elements: {text: (x, y)}
text_data = {
    "50% OFF": (50, 50),
    "Limited Time Offer": (50, 120),
    "Shop Now!": (50, 190)
}

# Platform sizes: [(width, height)]
platforms = [
    (1200, 600),  # Web banner
    (1080, 1080),  # Social media ad
    (600, 600)     # Product thumbnail
]

# Generate hero images
create_hero_image(product_image, background_image, output_folder, text_data, platforms)
